In [7]:
from fastcore.xtras import load_pickle
import pandas as pd 
import os 
from glob import glob
from pathlib import Path
from scipy.stats import sem
import matplotlib.pyplot as plt
import pycm 

FOLDER = 'out'

In [8]:
if not os.path.exists(FOLDER):
    raise FileExistsError('Cannot find folder {}'.format(FOLDER))

all_res = glob(f'{FOLDER}/*.pkl')
print('Pickles found in {}: {}'.format(FOLDER, len(all_res)))

Pickles found in out: 23


In [9]:

compiled_res_list = []
conf_mats = []

for results in all_res:
    res_full = load_pickle(results)
    res = res_full['results']
    summary = {
        'size': res_full['train_size'],
        'target': res_full['data_summary']['target'],
        'representation': res_full['data_summary']['representation'],
        'accuracy': res['accuracy'],
        #'f1_macro': res['f1_macro'],
        #'f1_micro': res['f1_micro'],
        'kappa': res['kappa'],
        'num_epochs': res_full['config']['tune_settings']['num_train_epochs'],
        'lr': res_full['config']['tune_settings']['learning_rate'],
        'bins': len(set(res['all_y_true'])),
        #'MCC': matthews_corrcoef(res['all_y_true'], res['all_y_pred'])
    }
    confusion_matrix = {
        'all_y_true':res['all_y_true'],
        'all_y_pred':res['all_y_pred']
    }
    conf_mats.append(confusion_matrix)
    compiled_res_list.append(summary)

In [10]:
compiled_res = pd.DataFrame(compiled_res_list)
grouped_res =compiled_res.groupby(['target', 'bins', 'num_epochs', 'representation', 'size']).agg(['mean', 'sem'])

In [11]:
grouped_res


accuracy            \
                                                        mean       sem   
target         bins num_epochs representation size                       
Diffusion_bool 2    10         A              25    0.506667  0.006667   
                    25         A              500   0.660000  0.023094   
                               AB             500   0.780000  0.030551   
                               ABC            500   0.780000  0.000000   
                               ABCD           500   0.780000  0.023094   
                               B              500   0.773333  0.017638   
                               mofid          500   0.480000  0.040000   
                               mofkey         500   0.526667  0.013333   

                                                       kappa            \
                                                        mean       sem   
target         bins num_epochs representation size                       
Diffusion_bool 2    10         A              25    0.013333  0.013333   
                    25         A              500   0.320000  0.046188   
                               AB             500   0.562614  0.062006   
                               ABC            500   0.560000  0.000000   
                               ABCD           500   0.560000  0.046188   
                               B              500   0.546667  0.035277   
                               mofid          500  -0.040000  0.080000   
                               mofkey         500   0.053333  0.026667   

                                                        lr       
                                                      mean  sem  
target         bins num_epochs representation size               
Diffusion_bool 2    10         A              25    0.0003  0.0  
                    25         A              500   0.0003  0.0  
                               AB             500   0.0003  0.0  
                               ABC            500   0.0003  0.0  
                               ABCD           500   0.0003  0.0  
                               B              500   0.0003  0.0  
                               mofid          500   0.0003  0.0  
                               mofkey         500   0.0003  0.0